In [ ]:
%matplotlib inline

import cv2
import numpy as np
import pandas as pd
from os import listdir

from utils import imshow

pd.options.display.html.border = 0
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
data_path = "./data/"
cars = [cv2.imread(data_path+img, 1) for img in listdir(data_path)]
imshow(cars[0])

In [ ]:
def median_filter(imgs: list[np.ndarray]) -> np.ndarray:
    imgs_stacked = np.stack(imgs, axis=2)
    imgs_gray = [cv2.cvtColor(car, cv2.COLOR_BGR2GRAY) for car in imgs]
    imgs_gray_stacked = np.stack(imgs_gray,axis=2)
    imgs_median_arg = np.argsort(imgs_gray_stacked,axis=2)[:,:,1]

    m, n, _ = imgs[0].shape
    i, j = np.indices((m, n))
    imgs_color = imgs_stacked[i, j, imgs_median_arg[i, j]]
    
    return imgs_color
        

In [ ]:
cars_median = median_filter(cars)

imshow(cars_median)

The result of the median filter is very good, however there's some car artifacts on the right where, there are cars in 2 pictures.

In [ ]:
def subtract_imgs(img_1: np.ndarray, img_2: np.ndarray) -> np.ndarray:
    img_1_gray = cv2.cvtColor(img_1, cv2.COLOR_BGR2GRAY)
    img_2_gray = cv2.cvtColor(img_2, cv2.COLOR_BGR2GRAY)
    img_diff = cv2.absdiff(img_1_gray, img_2_gray)
    return img_diff

In [ ]:
def subtract_mask_replace(img_to_replace: np.ndarray, img_to_mask: np.ndarray, img_1: np.ndarray, img_2: np.ndarray, erode: bool = False, thresh: int = 25, space: int = 15) -> np.ndarray:
    img_sub = subtract_imgs(img_1, img_2)
    img_thresh = cv2.threshold(img_sub, thresh, 255, cv2.THRESH_BINARY)[1]
    img_space_closed = cv2.morphologyEx(img_thresh, cv2.MORPH_CLOSE, np.ones((space, space),np.uint8))

    if erode:
        struct = np.ones((2,2),np.int8)
        struct[0, :] = -10
        img_space_closed = cv2.erode(img_space_closed, struct, iterations=10)

    img_masked = cv2.bitwise_and(img_to_mask, img_to_mask, mask=img_space_closed)
    img_thresh_stacked = np.stack([img_space_closed]*3, axis=2)
    img_replaced = np.where(img_thresh_stacked == 255, img_masked, img_to_replace)
    return img_replaced


cars_masked = subtract_mask_replace(cars_median, cars[2], cars[0], cars[1], erode=False, thresh=25, space=15)
cars_masked_eroded = subtract_mask_replace(cars_median, cars[2], cars[0], cars[1], erode=True, thresh=25, space=15)
cars_masked_2 = subtract_mask_replace(cars_masked_eroded, cars[1], cars[2], cars_masked_eroded, erode=True, thresh=25, space=15)
cars_masked_3 = subtract_mask_replace(cars_masked, cars[1], cars_masked, cars_masked_eroded, erode=True, thresh=25, space=15)
cars_median_2 = median_filter([cars_masked_2, cars[1], cars_masked_3])
cars_masked_4 = subtract_mask_replace(cars_median_2, cars[1], cars_median_2, cars_masked, thresh=15, space=10)
imshow(cars_masked)
imshow(cars_masked_eroded)
imshow(cars_masked_2)
imshow(cars_masked_3)
imshow(cars_median_2)
imshow(cars_masked_4)